In [1]:
# 15m 收在下布林，且下影線大於實體Ｋ及上影線
# ! conda install -c conda-forge ta --yes

In [2]:
import websocket
import requests
import json
import pandas as pd
import ta
import matplotlib.pyplot as plt
import math
import pytz
import datetime as dt
import numpy as np
import time

In [3]:
timezone = 8
endpoint = 'wss://stream.binance.com:9443/ws'
symbol = 'ethusdt'
symbol_C = symbol.upper()
interval = '15m'

# start epoch till now
start_time = 1675989807000
end_time = round(time.time() * 1000)
# end_time = 1672556207000

# step between timestamps in milliseconds
step = 60000 * 3600 # 15min

In [4]:
def get_historical(symbol, interval, start_time, end_time, step):
    
    rawdf = pd.DataFrame()
    
    url = "https://api.binance.com/api/v3/klines"
    
    for timestamp in range(start_time, end_time, step):
        params = {"symbol": symbol_C,
                  "interval": interval,
                  "startTime": timestamp,
                  "endTime": timestamp + step}
        response = requests.get(url, params=params).json()
        out = pd.DataFrame(response, columns = ["Open time", "Open", "High", "Low", "Close",
                                               "Volume", "Close_Time", "Quote asset volume",
                                               "Number of trades", "Taker buy base asset volume",
                                               "Taker buy quote asset volume", "Ignore"])
        rawdf = pd.concat([rawdf, out], axis = 0)
    
    rawdf = rawdf[['Close_Time', 'Open', 'Close', "High", "Low", 'Volume']]
    convert_dict = {'Close_Time': float, 'Open': float, 'Close': float, "High": float, "Low": float, 'Volume': float}
    rawdf = rawdf.astype(convert_dict)

    rawdf['Close_Time'] = pd.to_datetime(rawdf['Close_Time'], unit = 'ms')
    rawdf['Close_Time'] = rawdf['Close_Time'] + pd.Timedelta(hours=timezone)
    rawdf['Close_Time'] = rawdf['Close_Time'].dt.strftime('%Y-%m-%d %H:%M:%S')
    
    rawdf = rawdf.reset_index(drop=True)
    
    return rawdf

In [5]:
rawdf = get_historical(symbol, interval, start_time, end_time, step)
print(rawdf)

              Close_Time     Open    Close     High      Low     Volume
0    2023-02-10 08:59:59  1552.01  1548.52  1552.01  1545.66  6673.8862
1    2023-02-10 09:14:59  1548.51  1552.63  1553.16  1545.47  6041.7708
2    2023-02-10 09:29:59  1552.63  1545.00  1552.63  1544.04  8270.4900
3    2023-02-10 09:44:59  1545.00  1548.18  1549.98  1544.64  3995.4669
4    2023-02-10 09:59:59  1548.19  1549.84  1550.27  1547.79  5160.2732
..                   ...      ...      ...      ...      ...        ...
816  2023-02-18 20:59:59  1694.95  1698.34  1699.49  1694.70  3690.0684
817  2023-02-18 21:14:59  1698.35  1696.45  1698.35  1695.22  3063.5568
818  2023-02-18 21:29:59  1696.45  1697.13  1699.43  1695.84  2222.4397
819  2023-02-18 21:44:59  1697.13  1696.24  1697.45  1695.21  1794.8177
820  2023-02-18 21:59:59  1696.24  1693.67  1696.39  1693.03  2288.7764

[821 rows x 6 columns]


In [54]:
df = rawdf.copy()

In [55]:
def indicators(df):
#     sma
    sma_int = 30
    df['sma'] = ta.trend.sma_indicator(df.Close, window=sma_int)
#     bband
    bb_int = 30
    bb_dev = 2
    bb = ta.volatility.BollingerBands(df['Close'], window=bb_int, window_dev=bb_dev)
    df['bb_u'] = bb.bollinger_hband()
    df['bb_m'] = bb.bollinger_mavg()
    df['bb_l'] = bb.bollinger_lband()
#     rsi
    rsi_int = 14
    df['rsi'] = ta.momentum.RSIIndicator(df['Close'], window = rsi_int).rsi()
    return df

In [56]:
indicators(df)

,Close_Time,Open,Close,High,Low,Volume,sma,bb_u,bb_m,bb_l,rsi
0,2023-02-10 08:59:59,1552.01,1548.52,1552.01,1545.66,6673.8862,NaN,NaN,NaN,NaN,NaN
1,2023-02-10 09:14:59,1548.51,1552.63,1553.16,1545.47,6041.7708,NaN,NaN,NaN,NaN,NaN
2,2023-02-10 09:29:59,1552.63,1545.00,1552.63,1544.04,8270.4900,NaN,NaN,NaN,NaN,NaN
3,2023-02-10 09:44:59,1545.00,1548.18,1549.98,1544.64,3995.4669,NaN,NaN,NaN,NaN,NaN
4,2023-02-10 09:59:59,1548.19,1549.84,1550.27,1547.79,5160.2732,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
816,2023-02-18 20:59:59,1694.95,1698.34,1699.49,1694.70,3690.0684,1693.555333,1699.500054,1693.555333,1687.610613,60.083869
817,2023-02-18 21:14:59,1698.35,1696.45,1698.35,1695.22,3063.5568,1693.518333,1699.376074,1693.518333,1687.660593,55.541916
818,2023-02-18 21:29:59,1696.45,1697.13,1699.43,1695.84,2222.4397,1693.474667,1699.203105,1693.474667,1687.746228,56.807029
819,2023-02-18 21:44:59,1697.13,1696.24,1697.45,1695.21,1794.8177,1693.444667,1699.105031,1693.444667,1687.784302,54.616406


In [57]:
def conditions(df):

    # c1 下影線 大於 實體Ｋ & 下影線 大於 上影線
    # c2 收盤 小於 布林下

    for index, row in df.iterrows():
        # c1 red candle
        df['c1_1'] = (abs(df['Low'] - df['Close']) > abs(df['Open'] - df['Close'])) & (abs(df['Low'] - df['Close']) > abs(df['High'] - df['Open']))
        # c1 green candle
        df['c1_2'] = (abs(df['Low'] - df['Open']) > abs(df['Open'] - df['Close'])) & (abs(df['Low'] - df['Open']) > abs(df['High'] - df['Close']))
        # c2
        df['c2'] = df['Close'] < df['bb_l']

    # 條件達成
    df['signal'] = False
    df.loc[df.c1_1 & df.c1_2 & df.c2, 'signal'] = True


    # 下一根進場
    df['openbuy'] = False
    for i in range(len(df) - 1):
        if df.loc[i, 'signal'] == True:
            df.loc[i + 1, 'openbuy'] = True
    
    return df

In [58]:
conditions(df)

,Close_Time,Open,Close,High,Low,Volume,sma,bb_u,bb_m,bb_l,rsi,c1_1,c1_2,c2,signal,openbuy
0,2023-02-10 08:59:59,1552.01,1548.52,1552.01,1545.66,6673.8862,NaN,NaN,NaN,NaN,NaN,False,True,False,False,False
1,2023-02-10 09:14:59,1548.51,1552.63,1553.16,1545.47,6041.7708,NaN,NaN,NaN,NaN,NaN,True,False,False,False,False
2,2023-02-10 09:29:59,1552.63,1545.00,1552.63,1544.04,8270.4900,NaN,NaN,NaN,NaN,NaN,False,True,False,False,False
3,2023-02-10 09:44:59,1545.00,1548.18,1549.98,1544.64,3995.4669,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False
4,2023-02-10 09:59:59,1548.19,1549.84,1550.27,1547.79,5160.2732,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
816,2023-02-18 20:59:59,1694.95,1698.34,1699.49,1694.70,3690.0684,1693.555333,1699.500054,1693.555333,1687.610613,60.083869,False,False,False,False,False
817,2023-02-18 21:14:59,1698.35,1696.45,1698.35,1695.22,3063.5568,1693.518333,1699.376074,1693.518333,1687.660593,55.541916,False,True,False,False,False
818,2023-02-18 21:29:59,1696.45,1697.13,1699.43,1695.84,2222.4397,1693.474667,1699.203105,1693.474667,1687.746228,56.807029,False,False,False,False,False
819,2023-02-18 21:44:59,1697.13,1696.24,1697.45,1695.21,1794.8177,1693.444667,1699.105031,1693.444667,1687.784302,54.616406,True,True,False,False,False


In [59]:
buydf = conditions(df)

# 進出場

in_position = False
stop_loss = np.nan

for index, row in buydf.iterrows():

    if index == 0:
        continue
        
        # set entry price (前一根Ｋ收盤價進場)
        # set stop loss (前一根Ｋ線低點停損)
        # 進場
    elif buydf.at[index, 'openbuy'] == True:
        close_val = buydf['Close']
        low_val = buydf['Low']
        buydf.at[index, 'entry_p'] = close_val.shift(1).at[index]
        buydf.at[index, 'stop_loss'] = low_val.shift(1).at[index]
        buydf.at[index, 'position'] = 'Buy'
        in_position = True
        stop_loss = low_val.shift(1).at[index]
    

    # 吃筍
    #-----------------------------重要-----------------------------
    # 若用 if 寫，則有可能入場馬上吃筍，若用 elif 則一個 iteration 只會執行一次
    elif in_position == True and buydf.at[index, 'Close'] < stop_loss:
        buydf.at[index, 'position'] = 'Stop'
        in_position = False
        stop_loss = np.nan

    # set take profit （高點碰到上布林，即刻出場）
    elif buydf.at[index, 'High'] >= buydf.at[index, 'bb_u'] and in_position == True:

        buydf.at[index, 'position'] = 'Sell'
        in_position = False
        stop_loss = np.nan


# 過濾有訊號或事件發生的Ｋ線
buydf = buydf[(buydf['openbuy'] == True) |
              (buydf['signal'] == True) | 
              (buydf['position'] == 'Buy') |
              (buydf['position'] == 'Sell') |
              (buydf['position'] == 'Stop')]


print(buydf)


              Close_Time     Open    Close     High      Low      Volume  \
60   2023-02-10 23:59:59  1525.90  1522.36  1525.90  1506.00  20219.5380   
61   2023-02-11 00:14:59  1522.35  1526.56  1534.00  1520.61  11901.9825   
64   2023-02-11 00:59:59  1519.19  1519.78  1523.85  1514.62   6598.5711   
65   2023-02-11 01:14:59  1519.77  1521.01  1524.12  1519.18   4775.7314   
83   2023-02-11 05:44:59  1520.12  1500.97  1521.51  1490.00  27091.7954   
167  2023-02-12 02:44:59  1515.52  1513.20  1516.07  1505.00  13062.7308   
168  2023-02-12 02:59:59  1513.20  1514.77  1516.46  1512.09   2547.0834   
179  2023-02-12 05:44:59  1521.77  1525.95  1526.39  1521.09   2362.0129   
240  2023-02-12 20:59:59  1527.23  1525.58  1529.46  1521.12   7862.0612   
241  2023-02-12 21:14:59  1525.57  1527.39  1528.05  1524.03   3203.5533   
243  2023-02-12 21:44:59  1524.48  1524.19  1525.67  1521.43   3221.2165   
244  2023-02-12 21:59:59  1524.19  1526.26  1527.00  1524.18   1925.4713   
250  2023-02

In [60]:
# 部位回測

posdf = buydf.copy()
posdf = posdf.reset_index(drop = True)
posdf = posdf[(posdf['position'] == 'Buy') |
              (posdf['position'] == 'Sell') |
              (posdf['position'] == 'Stop')]

# 一次進場多少單位
pos_size = 1

col = ['Close_Time', 'Open', 'Close', 'High', 'Low', 'sma', 'bb_u', 'bb_l', 'rsi', 'position','entry_p', 'stop_loss']
pos = posdf[col]
pos = pos.reset_index(drop = True)


for index, row in pos.iterrows():
    
    current_pos = 0
    
    # 進場
    if pos.at[index, 'position'] == 'Buy':
        pos.at[index, 'size'] = pos_size
        pos.exit_p = np.nan
    
    # 出場
    if pos.at[index, 'position'] == 'Sell' or pos.at[index, 'position'] == 'Stop':
        
        #-----------------------------重要-----------------------------
        # 實戰需即刻出場
        
        # 停利：打到上布林
        if pos.at[index, 'position'] == 'Sell':
            pos.at[index, 'exit_p'] = pos.at[index, 'bb_u'] * 1

        # 停損：打到進場停損點（往回跌代，直到最近的'Buy'及其'stop_loss'）
        if pos.at[index, 'position'] == 'Stop':
            for i in range(index -1, -1, -1):
                if pos.at[i, 'position'] == 'Buy':
                    pos.at[index, 'exit_p'] = pos.at[i, 'stop_loss']
                break

        # 計算每次出場部位大小（每次出場皆清倉）
        for i in range(index -1, -1, -1):
            if pos.at[i, 'position'] == 'Buy':
                current_pos += pos.at[i, 'size']
                if i == 0:
                    pos.at[index, 'size'] = -current_pos
                else:
                    continue
            else:
                pos.at[index, 'size'] = -current_pos
                current_pos = 0
                break


# 計算部位價值
for index, row in pos.iterrows():
    if pos.at[index, 'position'] == 'Buy':
        pos.at[index, 'amt'] = round(pos.at[index, 'size'] * pos.at[index, 'entry_p'], 4)
    elif pos.at[index, 'position'] == 'Sell' or pos.at[index, 'position'] == 'Stop':
        pos.at[index, 'amt'] = round(pos.at[index, 'size'] * pos.at[index, 'exit_p'], 4)


# 若最後一筆為 Buy，移除該單，迭代驗證
for index, row in pos.iloc[::-1].iterrows():
    if row['position'] == 'Buy':
        pos = pos.drop(index)
    else:
        break
        
print(pos)
pos.to_csv('pos.csv')


# 手續費、滑點、價差
fee = 0.075 / 100
amt_abs_sum = pos.amt.abs().sum()
ttl_fee = amt_abs_sum * fee

# 損益
leverage = 10
ttl_profit = -pos.amt.sum() - ttl_fee


# 計算進場最大部位來代表總進場成本，以計算報酬率
pos['consec_entry'] = (pos['position'] != pos['position'].shift()).cumsum()
group_consec = pos.groupby('consec_entry').apply(lambda x:x.loc[x['position'] == 'Buy', 'amt' ].sum())
max_entry = group_consec.max()

profit_per = "{:.2f}%".format(ttl_profit / (max_entry/leverage) * 100)


result = {'Profit': [round(ttl_profit, 2)],
          'Fee': [round(ttl_fee, 2)],
          'Max_Entry': [round(max_entry, 2)],
          'Profit_%': [profit_per]}

result_df = pd.DataFrame(result)

print(result_df)



             Close_Time     Open    Close     High      Low          sma  \
0   2023-02-11 00:14:59  1522.35  1526.56  1534.00  1520.61  1540.695667   
1   2023-02-11 01:14:59  1519.77  1521.01  1524.12  1519.18  1536.700333   
2   2023-02-11 05:44:59  1520.12  1500.97  1521.51  1490.00  1527.183000   
3   2023-02-12 02:59:59  1513.20  1514.77  1516.46  1512.09  1519.946000   
4   2023-02-12 05:44:59  1521.77  1525.95  1526.39  1521.09  1519.675667   
5   2023-02-12 21:14:59  1525.57  1527.39  1528.05  1524.03  1532.992000   
6   2023-02-12 21:59:59  1524.19  1526.26  1527.00  1524.18  1532.133667   
7   2023-02-12 23:29:59  1535.46  1532.86  1541.00  1532.86  1532.002000   
8   2023-02-13 06:29:59  1502.65  1510.03  1512.15  1498.44  1534.966667   
9   2023-02-13 15:29:59  1520.74  1523.90  1526.10  1520.43  1515.385333   
10  2023-02-13 17:59:59  1484.21  1482.36  1492.51  1481.72  1512.949667   
11  2023-02-13 18:44:59  1474.80  1480.80  1480.90  1474.36  1509.735667   
12  2023-02-

In [53]:
plt.title(symbol + ' SMA_' + str(rate))
plt.xlabel('time')
plt.ylabel('Closing Prices')
plt.plot(df.Close, label='Closing Prices')
plt.plot(df.sma, label= str(rate) + ' SMA')
plt.plot(df.bb_u, label='Bollinger Up', c='g')
plt.plot(df.bb_l, label='Bollinger Down', c='r')
plt.legend()
plt.show()

NameError: name 'rate' is not defined